# Chapter 9. Getting Data

In [62]:
from __future__ import division
from collections import Counter
import math, random, csv, json
from bs4 import BeautifulSoup
import requests

In order to be a data scientist you need data (yes, really).  
In this chapter, we take a look at different ways of getting data into Python and into the right formats.

## stdin and stdout

If you run your Python scripts at the command line, you can *pipe* data through them using <code>sys.stdin</code> and <code>sys.stdout</code>.  
For example, here is a script that reads in lines of text and spits back out the ones that match a regular expression:

Here's a program that counts the lines it receives and then writes out the count:

You could then use these to count how many lines of a file contain numbers.  
In Windows, you would use:  

<code>type SomeFile.txt | python egrep.py "[0-9]" | python line_count.py</code>  

whereas in a Unix system you would use:  

<code>cat SomeFile.txt | python egrep.py "[0-9]" | python line_count.py</code>  

The | is the pipe character, which means "use the output of the left command as the input of the right command".  
You can build some pretty elaborate data-processing pipelines this way.

Similarly, here is a script that counts the words in its input and writes out the most common ones:

In [63]:
cat lorem_ipsum.txt | python most_common_words.py 10

7	eget
6	non
6	in
5	sed
5	ut
4	neque
4	vel
4	et
4	nulla
4	ullamcorper


**Note**  
There are plenty of Unix command-line tools that can also be used, such as <code>grep</code> and <code>egrep</code>, that are probably preferable to building your own from scratch.

## Reading Files